# **Feature Engineering Tutorial with Titanic**

Source: https://www.kaggle.com/kernels/scriptcontent/13445201/download

> ## **0. Introduction**

> **tl;dr** Created many features by grouping and used a simple ensemble model.

> I decided to write this kernel because **Titanic: Machine Learning from Disaster** is one of my favorite competitions on Kaggle. This is a basic level kernel which focuses on **Exploratory Data Analysis** and **Feature Engineering**. A lot of people start Kaggle with this competition and they get lost in extremely long tutorial kernels. This is a relatively short kernel compared to others. I hope this will be a guide for starters and inspire them with Feature Engineering ideas.

> **Titanic: Machine Learning from Disaster** is a great competition to apply domain knowledge for feature engineering, so I made a research and learned a lot about Titanic. There are many secrets to be revealed beneath the Titanic dataset. I tried to find out some of those secret factors that had affected the survival of passengers when the Titanic was sinking. I believe there are features still waiting to be discovered. 

> This kernel has **3** main parts; **Exploratory Data Analysis**, **Feature Engineering** and **Machine Learning**, and it can achieve top **5%** LB score with a simple tuned Random Forest Classifier. It takes 30 seconds to run whole notebook. I have been revisiting this kernel from time to time. If you have any idea that might improve this kernel, please be sure to comment or fork and experiment as you like.

> **TODO;**
* The continuous features (`Fare` and `Age`) are binned based on their quantiles. In order to increase the information gain via binning, entropy based binning can be implemented. This way the most homogeneous groups in those features can be revealed.

>> ### **0.1. Libraries**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import string
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

SEED = 42

>> ### **0.2. Loading the Dataset**
* Training set has **891** rows and test set has **418** rows.
* Training set have **12** features and test set have **11** features.
* One extra feature in the training set is `Survived` feature, which is the class of a row.

In [ ]:
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set on axis 0
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

def divide_df(all_data):
    # Returns divided dfs of training and test set
    return all_data.loc[:890], all_data.loc[891:].drop(['Survived'], axis=1)

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_all = concat_df(df_train, df_test)

df_train.name = 'Training Set'
df_test.name = 'Test Set'
df_all.name = 'All Set' 

dfs = [df_train, df_test]

print('Number of Training Examples = {}'.format(df_train.shape[0]))
print('Number of Test Examples = {}\n'.format(df_test.shape[0]))
print('Training X Shape = {}'.format(df_train.shape))
print('Training y Shape = {}\n'.format(df_train['Survived'].shape[0]))
print('Test X Shape = {}'.format(df_test.shape))
print('Test y Shape = {}\n'.format(df_test.shape[0]))
print(df_train.columns)
print(df_test.columns)

> ## **1. Exploratory Data Analysis**

>> ### **1.1. Overview**
* `PassengerId` feature is the unique id of the row and it doesn't have any effect on `Survived`.
* `Survived` feature is binary (**0** or **1**); 
    - **1 = Survived**
    - **0 = Not Survived**
* `Pclass` (Passenger Class) feature is the socio-economic status of the passenger. It is a categorical ordinal feature which has **3** unique values (**1**,  **2 **or **3**);
    - **1 = Upper Class**
    - **2 = Middle Class**
    - **3 = Lower Class**
* `Name`, `Sex` and `Age` features are self-explanatory.
* `SibSp` feature is the total number of the passengers' siblings and spouse.
* `Parch` feature is the total number of the passengers' parents and children.
* `Ticket` feature is the ticket number of the passenger.
* `Fare` feature is the passenger fare.
* `Cabin` feature is the cabin number of the passenger.
* `Embarked` is port of embarkation. It is a categorical feature and it has **3** unique values (**C**, **Q** or **S**);
    - **C = Cherbourg**
    - **Q = Queenstown**
    - **S = Southampton**

In [ ]:
print(df_train.info())
df_train.sample(3)

In [ ]:
print(df_test.info())
df_test.sample(3)

>> ### **1.2. Dealing with Missing Values**
As seen from below, there are some columns have missing values. The `display_missing` function shows the sum of missing values in each column in both training and test set.
* Training set have missing values in `Age`, `Cabin` and `Embarked` columns
* Test set have missing values in `Age`, `Cabin` and `Fare` columns

>> It is convenient to merge training and test set before dealing with missing values, otherwise the filled data tends to overfit training set. The amount of missing values in `Age`, `Embarked` and `Fare` features are relatively smaller compared to the total samples, but roughly **80%** of the `Cabin` feature is missing in both training and test set. Missing values in `Age`, `Embarked` and `Fare` features can be filled with descriptive statistical measures, but that wouldn't work for `Cabin` feature.

In [ ]:
def display_missing(df):    
    for col in df.columns.tolist():          
        print('{} column missing values: {}'.format(col, df[col].isnull().sum()))
    print('\n')
    
for df in dfs:
    print('{}'.format(df.name))
    display_missing(df)

>>> #### **1.2.1. Age**
Missing values in `Age` feature are filled with the median age, but using the median age of the whole data set is not a good choice. Median age of a group is much better because the new values would be more informative. Median age of `Pclass` groups is the best choice because of its high correlation with `Age` **(0.408106)** and `Survived` **(0.338481)** features, and it has only **3** unique values. It also makes more sense to group ages by passenger classes instead of other features.

In [ ]:
df_all_corr = df_all.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_all_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_all_corr[df_all_corr['Feature 1'] == 'Age']

>>> In order to be more accurate, `Sex` feature is used as the second level of `groupby`. As seen from below, the `Pclass` and `Sex` groups have distinct median `Age` values. When passenger class increases, the median age for both males and females also increases. However, females tend to have slightly lower median `Age` compared to males. The median ages below are used for filling the missing values in `Age` feature.

In [ ]:
age_by_pclass_sex = df_all.groupby(['Sex', 'Pclass']).median()['Age']

for pclass in range(1, 4):
    for sex in ['female', 'male']:
        print('Median age of Pclass {} {}s: {}'.format(pclass, sex, age_by_pclass_sex[sex][pclass]))
print('Median age of all passengers: {}'.format(df_all['Age'].median()))

df_all['Age'] = df_all.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))

>>> #### **1.2.2. Embarked**
`Embarked` is a categorical feature and there are only **2** missing values in the whole dataset. Both of those passengers are female, upper class and they have the same ticket number. This means, they know each other and embarked from the same port together. The mode `Embarked` value for an upper class female passenger is **C (Cherbourg)**, but this doesn't necessarily mean that they embarked from that port. 

>>> When I googled **Stone, Mrs. George Nelson (Martha Evelyn)**, I found that she embarked from **S (Southampton)** with her maid **Amelie Icard**, in this page [Martha Evelyn Stone: Titanic Survivor](https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html). 

>>> *Mrs Stone boarded the Titanic in Southampton on 10 April 1912 and was travelling in first class with her maid Amelie Icard. She occupied cabin B-28.* This was the information we were looking for and case closed for `Embarked` feature.

In [ ]:
df_all[df_all['Embarked'].isnull()]

In [ ]:
df_all['Embarked'] = df_all['Embarked'].fillna('S')

>>> #### **1.2.3. Fare**
There is only one passenger with missing `Fare` value. We can assume that `Fare` is related to family size (`Parch` and `SibSp`) and `Pclass` features. Median `Fare` value of a male with a third class ticket and no family is a logical choice to fill the missing value.

In [ ]:
df_all[df_all['Fare'].isnull()]

In [ ]:
med_fare = df_all.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()[3][0][0]
df_all['Fare'] = df_all['Fare'].fillna(med_fare)

>>> #### **1.2.4. Cabin**
>>> The `Cabin` feature is little bit tricky and it needs further exploration. The large portion of the `Cabin` feature is missing and the feature itself can't be ignored completely because some cabins might have higher survival rates. It turns out to be the first letter of the `Cabin` values are the decks in which the cabins are located. Those decks were mainly separated for one passenger class, but some of them were used by multiple passenger classes.
![alt text](https://vignette.wikia.nocookie.net/titanic/images/f/f9/Titanic_side_plan.png/revision/latest?cb=20180322183733)
* On the Boat Deck there were **6** rooms labeled as **T, U, W, X, Y, Z**, but only the **T** cabin is present in the dataset.
* **A**, **B** and **C** decks were only for 1st class passengers.
* **D** and **E** decks were for all classes.
* **F** and **G** decks were for both 2nd and 3rd class passengers.
* From going **A** to **G**, the distance to the staircase increases, which might be a factor of survival. If it is true, then the new `Deck` feature is ordinal.

In [ ]:
# Creating Deck column from the first letter of the Cabin column (M stands for Missing)
df_all['Deck'] = df_all['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')

df_all_decks = df_all.groupby(['Deck', 'Pclass']).count().drop(columns=['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 
                                                                        'Fare', 'Embarked', 'Cabin', 'PassengerId', 'Ticket']).rename(columns={'Name': 'Count'}).transpose()

def get_pclass_dist(df):
    
    # Creating a dictionary for every passenger class count in every deck
    deck_counts = {'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}, 'F': {}, 'G': {}, 'M': {}, 'T': {}}
    decks = df.columns.levels[0]    
    
    for deck in decks:
        for pclass in range(1, 4):
            try:
                count = df[deck][pclass][0]
                deck_counts[deck][pclass] = count 
            except KeyError:
                deck_counts[deck][pclass] = 0
                
    df_decks = pd.DataFrame(deck_counts)    
    deck_percentages = {}

    # Creating a dictionary for every passenger class percentage in every deck
    for col in df_decks.columns:
        deck_percentages[col] = [(count / df_decks[col].sum()) * 100 for count in df_decks[col]]
        
    return deck_counts, deck_percentages

def display_pclass_dist(percentages):
    
    df_percentages = pd.DataFrame(percentages).transpose()
    deck_names = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'M', 'T')
    bar_count = np.arange(len(deck_names))  
    bar_width = 0.85
    
    pclass1 = df_percentages[0]
    pclass2 = df_percentages[1]
    pclass3 = df_percentages[2]
    
    plt.figure(figsize=(20, 10))
    plt.bar(bar_count, pclass1, color='#b5ffb9', edgecolor='white', width=bar_width, label='Passenger Class 1')
    plt.bar(bar_count, pclass2, bottom=pclass1, color='#f9bc86', edgecolor='white', width=bar_width, label='Passenger Class 2')
    plt.bar(bar_count, pclass3, bottom=pclass1 + pclass2, color='#a3acff', edgecolor='white', width=bar_width, label='Passenger Class 3')

    plt.xticks(bar_count, deck_names)
    plt.xlabel('Deck', size=15, labelpad=20)
    plt.ylabel('Passenger Class Percentage', size=15, labelpad=20)
    
    plt.tick_params(axis='x', labelsize=15)
    plt.tick_params(axis='y', labelsize=15)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), prop={'size': 15})
    plt.title('Passenger Class Distribution in Decks', size=18, y=1.05)    
    plt.show()    

all_deck_count, all_deck_per = get_pclass_dist(df_all_decks)
display_pclass_dist(all_deck_per)

>>> * **100%** of **A**, **B** and **C** decks are 1st class passengers.
* Deck **D** has **87%** 1st class and **13%** 2nd class passengers.
* Deck **E** has **83%** 1st class, **10%** 2nd class and **7%** 3rd class passengers.
* Deck **F** has **62%** 2nd class and **38%** 3rd class passengers.
* **100%** of **G** deck are 3rd class passengers.
* There is one person on the boat deck in the **T** cabin and he is a 1st class passenger. **T** cabin passenger has the closest resemblance to **A** deck passengers, so he is grouped in **A** deck.
* Passengers labeled as **M** are the missing values in the `Cabin` feature. I don't think it is possible to find those passengers' real `Deck`. I decided to use **M** like a deck itself.

In [ ]:
# Passenger in the T deck is changed to A
idx = df_all[df_all['Deck'] == 'T'].index
df_all.loc[idx, 'Deck'] = 'A'

In [ ]:
df_all_decks_survived = df_all.groupby(['Deck', 'Survived']).count().drop(columns=['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 
                                                                                   'Embarked', 'Pclass', 'Cabin', 'PassengerId', 'Ticket']).rename(columns={'Name':'Count'}).transpose()

def get_survived_dist(df):
    
    # Creating a dictionary for every survival count in every deck
    surv_counts = {'A':{}, 'B':{}, 'C':{}, 'D':{}, 'E':{}, 'F':{}, 'G':{}, 'M':{}}
    decks = df.columns.levels[0]    

    for deck in decks:
        for survive in range(0, 2):
            surv_counts[deck][survive] = df[deck][survive][0]
            
    df_surv = pd.DataFrame(surv_counts)
    surv_percentages = {}

    for col in df_surv.columns:
        surv_percentages[col] = [(count / df_surv[col].sum()) * 100 for count in df_surv[col]]
        
    return surv_counts, surv_percentages

def display_surv_dist(percentages):
    
    df_survived_percentages = pd.DataFrame(percentages).transpose()
    deck_names = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'M')
    bar_count = np.arange(len(deck_names))  
    bar_width = 0.85    

    not_survived = df_survived_percentages[0]
    survived = df_survived_percentages[1]
    
    plt.figure(figsize=(20, 10))
    plt.bar(bar_count, not_survived, color='#b5ffb9', edgecolor='white', width=bar_width, label="Not Survived")
    plt.bar(bar_count, survived, bottom=not_survived, color='#f9bc86', edgecolor='white', width=bar_width, label="Survived")

    plt.xticks(bar_count, deck_names)
    plt.xlabel('Deck', size=15, labelpad=20)
    plt.ylabel('Survival Percentage', size=15, labelpad=20)
    
    plt.tick_params(axis='x', labelsize=15)
    plt.tick_params(axis='y', labelsize=15)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), prop={'size': 15})
    plt.title('Survival Percentage in Decks', size=18, y=1.05)
    plt.show()

all_surv_count, all_surv_per = get_survived_dist(df_all_decks_survived)
display_surv_dist(all_surv_per)

>>> As I suspected, every deck has different survival rates and we can't discard this information. Deck **B**, **C**, **D** and **E** have the highest survival rates. Those decks are mostly occupied by 1st class passengers. **M** has the lowest survival rate which is mostly occupied by 2nd and 3rd class passengers. To conclude cabins used by 1st class passengers have higher survival rates than cabins used by 2nd and 3rd class passengers. In my opinion **M** (Missing `Cabin` values) has the lowest survival rate because they couldn't retrieve the cabin data of the victims. That's why I believe, labeling that group as **M** is a reasonable way to handle the missing data. It is a unique group with same characteristics. `Deck` feature has high-cardinality right now, so some of the values are grouped with each other depending on their resemblance.
* **A**, **B** and **C** decks are labeled as **ABC** because all of them have only 1st class passengers.
* **D** and **E** decks are labeled as **DE** because both of them have similar passenger class distribution and same survival rate.
* **F** and **G** decks are labeled as **FG** because of the former reason.
* **M** deck doesn't need to be grouped with other decks because it is very different from others and has the lowest survival rate.

In [ ]:
df_all['Deck'] = df_all['Deck'].replace(['A', 'B', 'C'], 'ABC')
df_all['Deck'] = df_all['Deck'].replace(['D', 'E'], 'DE')
df_all['Deck'] = df_all['Deck'].replace(['F', 'G'], 'FGM')

df_all['Deck'].value_counts()

In [ ]:
# Dropping the Cabin feature
df_all.drop(['Cabin'], inplace=True, axis=1)

df_train, df_test = divide_df(df_all)
dfs = [df_train, df_test]

for df in dfs:
    display_missing(df)

>> ### **1.3. Survival Distribution**
* **38.38%** (342/891) of the training set is **Class 1**.
* **61.62%** (549/891) of the training set is **Class 0**.

In [ ]:
survived = df_train['Survived'].value_counts()[1]
not_survived = df_train['Survived'].value_counts()[0]
survived_per = survived / df_train.shape[0] * 100
not_survived_per = not_survived / df_train.shape[0] * 100

print('{} of {} passengers survived and it is the {:.2f}% of the training set.'.format(survived, df_train.shape[0], survived_per))
print('{} of {} passengers didnt survive and it is the {:.2f}% of the training set.'.format(not_survived, df_train.shape[0], not_survived_per))

plt.figure(figsize=(10, 8))
sns.countplot(df_train['Survived'])

plt.xticks((0, 1), ['Not Survived ({0:.2f}%)'.format(not_survived_per), 'Survived ({0:.2f}%)'.format(survived_per)])
plt.tick_params(axis='x', labelsize=13)
plt.tick_params(axis='y', labelsize=13)
plt.xlabel('Survival', size=15, labelpad=15)
plt.ylabel('Passenger Count', size=15, labelpad=15)
plt.title('Training Set Survival Distribution', size=15, y=1.05)

plt.show()

>> ### **1.4. Correlations**
* Features are highly correlated with each other and dependent to each other.
* The highest correlation between features is **0.549500** in training set and **0.577147** in test set (between `Fare` and `Pclass`).
* The other features are also highly correlated. There are **9** correlations in training set and **6** correlations in test set that are higher than **0.1**.

In [ ]:
df_train_corr = df_train.drop(['PassengerId'], axis=1).corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_train_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_train_corr.drop(df_train_corr.iloc[1::2].index, inplace=True)
df_train_corr_nd = df_train_corr.drop(df_train_corr[df_train_corr['Correlation Coefficient'] == 1.0].index)

df_test_corr = df_test.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_test_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_test_corr.drop(df_test_corr.iloc[1::2].index, inplace=True)
df_test_corr_nd = df_test_corr.drop(df_test_corr[df_test_corr['Correlation Coefficient'] == 1.0].index)

In [ ]:
# Training set high correlations
corr = df_train_corr_nd['Correlation Coefficient'] > 0.1
df_train_corr_nd[corr]

In [ ]:
# Test set high correlations
corr = df_test_corr_nd['Correlation Coefficient'] > 0.1
df_test_corr_nd[corr]

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(20, 20))

sns.heatmap(df_train.drop(['PassengerId'], axis=1).corr(), ax=axs[0], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 14})
sns.heatmap(df_test.drop(['PassengerId'], axis=1).corr(), ax=axs[1], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 14})

for i in range(2):    
    axs[i].tick_params(axis='x', labelsize=14)
    axs[i].tick_params(axis='y', labelsize=14)
    
axs[0].set_title('Training Set Correlations', size=15)
axs[1].set_title('Test Set Correlations', size=15)

plt.show()

>> ### **1.5. Survival Distribution in Features**

>>> #### **1.5.1. Continuous Features**
* Both of the continuous features, `Age` and `Fare`, have good split points and spikes for a decision tree.
* Distribution of `Age` feature clearly shows that children younger than 15 has a higher survival rate than any of the other age groups.
* In the distribution of `Fare` feature, the survival rate is higher on distribution tails. The distribution also has positive skew because of the extreme outliers.
* One potential problem for both features is, the distribution of the spikes are bumpy in training set, but smooth in test set. The machine learning algorithm may not able to generalize to test set because of this reason.

In [ ]:
cont_features = ['Age', 'Fare']
surv = df_train['Survived'] == 1

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(20, 20))
plt.subplots_adjust(right=1.5)

for i, feature in enumerate(cont_features):    
    # Distribution of survival in feature
    sns.distplot(df_train[~surv][feature], label='Not Survived', hist=True, color='#e74c3c', ax=axs[0][i])
    sns.distplot(df_train[surv][feature], label='Survived', hist=True, color='#2ecc71', ax=axs[0][i])
    
    # Distribution of feature in dataset
    sns.distplot(df_train[feature], label='Training Set', hist=False, color='#e74c3c', ax=axs[1][i])
    sns.distplot(df_test[feature], label='Test Set', hist=False, color='#2ecc71', ax=axs[1][i])
    
    for j in range(2):        
        axs[i][j].tick_params(axis='x', labelsize=20)
        axs[i][j].tick_params(axis='y', labelsize=20)
    
    axs[0][i].legend(loc='upper right', prop={'size': 20})
    axs[0][i].set_xlabel('')
    axs[0][i].set_title('Distribution of Survival in {}'.format(feature), size=20, y=1.05)
    
    axs[1][i].legend(loc='upper right', prop={'size': 20})
    axs[1][i].set_xlabel('')
    
axs[1][0].set_title('Distribution of {} Feature'.format('Age'), size=20, y=1.05)
axs[1][1].set_title('Distribution of {} Feature'.format('Fare'), size=20, y=1.05)
        
plt.show()

>>> #### **1.5.2. Categorical Features**
* Every categorical feature has at least one class with high not-survived rate. Those classes will be very helpful to predict whether the passenger is a survivor or victim.
* Passengers boarded from **Southampton** has a lower survival rate unlike other ports.More than half of the passengers boarded from **Cherbourg** did survive.
* `Parch` and `SibSp` features show that passengers with one family member has a higher survival rate.
* Best categorical features are `Pclass` and `Sex` because they have the highest survived/not-survived ratio in a single class.

In [ ]:
cat_features = ['Embarked', 'Parch', 'Pclass', 'Sex', 'SibSp', 'Deck']

fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(20, 20))
plt.subplots_adjust(right=1.5, top=1.25)

for i, feature in enumerate(cat_features, 1):
    
    plt.subplot(2, 3, i)
    sns.countplot(x=feature, hue='Survived', data=df_train)
    
    plt.tick_params(axis='x', labelsize=20)
    plt.tick_params(axis='y', labelsize=20)
    plt.legend(['Not Survived', 'Survived'], loc='upper center', prop={'size': 18})
    plt.xlabel('{}'.format(feature), size=20, labelpad=15)
    plt.ylabel('Passenger Count', size=20, labelpad=15)
    plt.title('Count of Survival in {} Feature'.format(feature), size=20, y=1.05)

plt.show()

>>> #### **1.5.3. Ticket Frequency**
* There were too many `Ticket` values to analyze, so grouping them up by frequency makes things easier. **How is this feature different than family size (`Parch` and `SibSp`)?** Many passengers travelled along with a group. Those groups consist of friends, nannies, maids and etc. They werent counted as family, but they used the same ticket.
* According to the graph, groups with **2**,**3** and **4** members had a higher rate of survival.
* Passengers who travel alone has the lowest survival rate.
* After **4** group members, the survival rate decreases drastically.

In [ ]:
df_train['Ticket_Frequency'] = df_all.groupby('Ticket')['Ticket'].transform('count').values[:891]
df_test['Ticket_Frequency'] = df_all.groupby('Ticket')['Ticket'].transform('count').values[891:]

fig, axs = plt.subplots(figsize=(12, 9))
sns.countplot(x='Ticket_Frequency', hue='Survived', data=df_train)

plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 15})
plt.xlabel('Ticket Frequency', size=15, labelpad=20)
plt.ylabel('Passenger Count', size=15, labelpad=20)
plt.title('Count of Survival in {} Feature'.format('Ticket Frequency'), size=15, y=1.05)

plt.show()

>> ### **1.6. Conclusion**
* Most of the features are correlated with each other. This relationship can be used to create new features with feature transformation and interaction. Target encoding could be very useful as well because of the high correlations with `Survived` feature.
* Split points and spikes are very visible in the continuous feature. They can be captured easily with a decision tree, but neural networks may not be able to spot them.
* Categorical features have very distinct classes with different survival rates. Some of those informations might also be combined to make new features.
* Created **2** new features (`Deck` and `Ticket_Frequency`) and dropped **1** feature (`Cabin`) at the Exploratory Data Analysis part.

In [ ]:
df_all = concat_df(df_train, df_test)
df_all.head()

> ## **2. Feature Engineering**

>> ### **2.1. Binning the Continuous Features**

>>> #### **2.1.1. Fare**
* `Fare` feature is positively skewed and survival rate is extremely high on the right end.
* **13** quantile based bins are used for `Fare` feature. Even though it is too much, it provides decent amount of information gain.
* The groups in the left side of the graph has the lowest survival rate and the groups in the right side of the graph has the highest survival rate. This high survival rate was not visible in the distribution graph.
* There is also an unusual group **(15.742, 23.25]** with high survival rate that is captured in this process.

In [ ]:
df_all['Fare'] = pd.qcut(df_all['Fare'], 13)

fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Fare', hue='Survived', data=df_all)

plt.tick_params(axis='x', labelsize=10)
plt.tick_params(axis='y', labelsize=15)
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 15})
plt.xlabel('Fare', size=15, labelpad=20)
plt.ylabel('Passenger Count', size=15, labelpad=20)
plt.title('Count of Survival in {} Feature'.format('Fare'), size=15, y=1.05)

plt.show()

>>> #### **2.1.2. Age**
* `Age` feature has a normal distribution with some spikes and bumps.
* **10** quantile based bins are used for `Age` feature.
* The first bin has the highest survival rate and 4th bin has the lowest survival rate. Those were the spikes in the distribution.
* There is also an unusual group **(34.0, 40.0]** with high survival rate that is captured in this process.

In [ ]:
df_all['Age'] = pd.qcut(df_all['Age'], 10)

fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Age', hue='Survived', data=df_all)

plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 15})
plt.xlabel('Age', size=15, labelpad=20)
plt.ylabel('Passenger Count', size=15, labelpad=20)
plt.title('Count of Survival in {} Feature'.format('Age'), size=15, y=1.05)

plt.show()

>> ### **2.2. Family Size**
* `Family_Size` is created by adding `SibSp`, `Parch` and **1**. `SibSp` is the count of siblings and spouse, and `Parch` is the count of parents and children. Those columns are added to find the total size of families. Adding **1** at the end, is the passenger.
* Graphs have clearly shown that family size is a predictor of survival because different values have different survival rates.
* Family Size with **1** are labeled as **Alone**.
* Family Size with **2**, **3** and **4** are labeled as **Small**.
* Family Size with **5** and **6** are labeled as **Medium**.
* Family Size with **7**, **8** and **11** are labeled as **Large**.

In [ ]:
df_all['Family_Size'] = df_all['SibSp'] + df_all['Parch'] + 1

fig, axs = plt.subplots(figsize=(20, 20), ncols=2, nrows=2)
plt.subplots_adjust(right=1.5)

sns.barplot(x=df_all['Family_Size'].value_counts().index, y=df_all['Family_Size'].value_counts().values, ax=axs[0][0])
sns.countplot(x='Family_Size', hue='Survived', data=df_all, ax=axs[0][1])
axs[0][0].set_title('Family Size Feature Value Counts', size=20, y=1.05)
axs[0][1].set_title('Count of Survival in Family Size ', size=20, y=1.05)

family_map = {1: 'Alone', 2: 'Small', 3: 'Small', 4: 'Small', 5: 'Medium', 6: 'Medium', 7: 'Large', 8: 'Large', 11: 'Large'}
df_all['Family_Size_Grouped'] = df_all['Family_Size'].map(family_map)

sns.barplot(x=df_all['Family_Size_Grouped'].value_counts().index, y=df_all['Family_Size_Grouped'].value_counts().values, ax=axs[1][0])
sns.countplot(x='Family_Size_Grouped', hue='Survived', data=df_all, ax=axs[1][1])
axs[1][0].set_title('Family Size Feature Value Counts After Grouping', size=20, y=1.05)
axs[1][1].set_title('Count of Survival in Family Size After Grouping', size=20, y=1.05)

for i in range(2):
    axs[i][1].legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 20})
    for j in range(2):
        axs[i][j].tick_params(axis='x', labelsize=20)
        axs[i][j].tick_params(axis='y', labelsize=20)
        axs[i][j].set_xlabel('')
        axs[i][j].set_ylabel('')

plt.show()

>> ### **2.3. Title & Is Married**
* `Title` is created by extracting the prefix before the `Name` column values.
* `Is_Married` is a binary feature based on the **Mrs** title. **Mrs** title has the highest survival rate among other female titles. This title needs to be a feature itself because other female titles are grouped with each other.
* According to the graph below, there are many titles which are occuring very few times. Some of those titles doesn't seem correct and they need to be replaced.
* **Miss**, **Mrs**, **Ms**, **Mlle**, **Lady**, **Mme**, **the Countess**, **Dona** titles are replaced with **Miss/Mrs/Ms** because all of them are actually females. Values like **Mlle**, **Mme** and **Dona** are actually the name of the passengers, but they are classified as titles because `Name` feature is split by comma.
* **Dr**, **Col**, **Major**, **Jonkheer**, **Capt**, **Sir**, **Don** and **Rev** titles are replaced with **Dr/Military/Noble/Clergy** because those passengers have similar characteristics.
* **Master** is a unique title. It is given to male passengers below age **26**. They have the highest survival rate among all male groups.

In [ ]:
df_all['Title'] = df_all['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
df_all['Is_Married'] = 0
df_all['Is_Married'].loc[df_all['Title'] == 'Mrs'] = 1

fig, axs = plt.subplots(nrows=2, figsize=(20, 20))

sns.barplot(x=df_all['Title'].value_counts().index, y=df_all['Title'].value_counts().values, ax=axs[0])
axs[0].set_title('Title Feature Value Counts', size=20, y=1.05)

df_all['Title'] = df_all['Title'].replace(['Miss', 'Mrs','Ms', 'Mlle', 'Lady', 'Mme', 'the Countess', 'Dona'], 'Miss/Mrs/Ms')
df_all['Title'] = df_all['Title'].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'], 'Dr/Military/Noble/Clergy')

axs[0].tick_params(axis='x', labelsize=10)
axs[1].tick_params(axis='x', labelsize=15)

for i in range(2):    
    axs[i].tick_params(axis='y', labelsize=15)

sns.barplot(x=df_all['Title'].value_counts().index, y=df_all['Title'].value_counts().values, ax=axs[1])
axs[1].set_title('Title Feature Value Counts After Grouping', size=20, y=1.05)

plt.show()

>> ### **2.4. Survival Rate**
* The first part of the feature, `Family_Survival_Rate`, is shared by [**@volhaleusha**](https://www.kaggle.com/volhaleusha) in this kernel [**Titanic: Tutorial, Encoding, Feature Eng, 81.8%**](https://www.kaggle.com/volhaleusha/titanic-tutorial-encoding-feature-eng-81-8), and it increases LB score significantly. However, it doesn't take passengers travelling with groups into consideration. Those groups doesn't necessarily have to be families. In order to overcome this problem, `Ticket_Survival_Rate` is also calculated and averaged with `Family_Survival_Rate`.
* `extract_surname` function is used to extract the surname of the passenger from the `Name` feature. `Family` feature is created with the extracted surname. This is necessary for grouping the passengers in the same family. 
* Family survival rate is calculated from the families in training set since there is no `Survived` feature in test set.
* A list of family names that are occuring in both training and test set, is created. The survival rate is calculated for families in that list and with more than 1 member, and stored in `Family_Survival_Rate` feature.
* An extra binary feature `Family_Survival_Rate_NA` is created for families that are unique to the test set. This feature is also necessary because there is no way to calculate those families' survival rate. This feature implies that family survival rate is not applicable to that passenger because there is no way to retrieve survival rate.
* `Ticket_Survival_Rate` and `Ticket_Survival_Rate_NA` features are also created the same method.
* `Ticket_Survival_Rate` and `Family_Survival_Rate` are averaged and become `Survival_Rate`, and `Ticket_Survival_Rate_NA` and `Family_Survival_Rate_NA` are averaged and become `Survival_Rate_NA`.

In [ ]:
def extract_surname(data):    
    
    families = []
    
    for i in range(len(data)):        
        name = data.iloc[i]

        if '(' in name:
            name_no_bracket = name.split('(')[0] 
        else:
            name_no_bracket = name
            
        family = name_no_bracket.split(',')[0]
        title = name_no_bracket.split(',')[1].strip().split(' ')[0]
        
        for c in string.punctuation:
            family = family.replace(c, '').strip()
            
        families.append(family)
            
    return families

df_all['Family'] = extract_surname(df_all['Name'])
df_train = df_all.loc[:890]
df_test = df_all.loc[891:]
dfs = [df_train, df_test]

# Returning a list of families that are occuring in both training and test set
non_unique_families = [x for x in df_train['Family'].unique() if x in df_test['Family'].unique()]
df_family_survival_rate = df_train.groupby('Family')['Survived', 'Family','Family_Size'].median()

family_rates = {}

for i in range(len(df_family_survival_rate)):
    # Checking a family exists in both training and test set, and has members more than 1
    if df_family_survival_rate.index[i] in non_unique_families and df_family_survival_rate.iloc[i, 1] > 1:
        family_rates[df_family_survival_rate.index[i]] = df_family_survival_rate.iloc[i, 0]

In [ ]:
mean_survival_rate = np.mean(df_train['Survived'])

train_family_survival_rate = []
train_family_survival_rate_NA = []
test_family_survival_rate = []
test_family_survival_rate_NA = []

for i in range(len(df_train)):
    if df_train['Family'][i] in family_rates:
        train_family_survival_rate.append(family_rates[df_train['Family'][i]])
        train_family_survival_rate_NA.append(1)
    else:
        train_family_survival_rate.append(mean_survival_rate)
        train_family_survival_rate_NA.append(0)
        
for i in range(len(df_test)):
    if df_test['Family'].iloc[i] in family_rates:
        test_family_survival_rate.append(family_rates[df_test['Family'].iloc[i]])
        test_family_survival_rate_NA.append(1)
    else:
        test_family_survival_rate.append(mean_survival_rate)
        test_family_survival_rate_NA.append(0)
        
df_train['Family_Survival_Rate'] = train_family_survival_rate
df_train['Family_Survival_Rate_NA'] = train_family_survival_rate_NA
df_test['Family_Survival_Rate'] = test_family_survival_rate
df_test['Family_Survival_Rate_NA'] = test_family_survival_rate_NA

In [ ]:
# Returning a list of tickets that are occuring in both training and test set
non_unique_tickets = [x for x in df_train['Ticket'].unique() if x in df_test['Ticket'].unique()]
df_ticket_survival_rate = df_train.groupby('Ticket')['Survived', 'Ticket','Ticket_Frequency'].median()

ticket_rates = {}

for i in range(len(df_ticket_survival_rate)):
    # Checking a ticket exists in both training and test set, and has members more than 1
    if df_ticket_survival_rate.index[i] in non_unique_tickets and df_ticket_survival_rate.iloc[i, 1] > 1:
        ticket_rates[df_ticket_survival_rate.index[i]] = df_ticket_survival_rate.iloc[i, 0]
        
train_ticket_survival_rate = []
train_ticket_survival_rate_NA = []
test_ticket_survival_rate = []
test_ticket_survival_rate_NA = []

for i in range(len(df_train)):
    if df_train['Ticket'][i] in ticket_rates:
        train_ticket_survival_rate.append(ticket_rates[df_train['Ticket'][i]])
        train_ticket_survival_rate_NA.append(1)
    else:
        train_ticket_survival_rate.append(mean_survival_rate)
        train_ticket_survival_rate_NA.append(0)
        
for i in range(len(df_test)):
    if df_test['Ticket'].iloc[i] in ticket_rates:
        test_ticket_survival_rate.append(ticket_rates[df_test['Ticket'].iloc[i]])
        test_ticket_survival_rate_NA.append(1)
    else:
        test_ticket_survival_rate.append(mean_survival_rate)
        test_ticket_survival_rate_NA.append(0)
        
df_train['Ticket_Survival_Rate'] = train_ticket_survival_rate
df_train['Ticket_Survival_Rate_NA'] = train_ticket_survival_rate_NA
df_test['Ticket_Survival_Rate'] = test_ticket_survival_rate
df_test['Ticket_Survival_Rate_NA'] = test_ticket_survival_rate_NA

for df in [df_train, df_test]:
    df['Survival_Rate'] = (df['Ticket_Survival_Rate'] + df['Family_Survival_Rate']) / 2
    df['Survival_Rate_NA'] = (df['Ticket_Survival_Rate_NA'] + df['Family_Survival_Rate_NA']) / 2
    

>> ### **2.5. Feature Transformation**

>>> #### **2.5.1. Label Encoding the Non-Numerical Features**
`Embarked`, `Sex`, `Deck` , `Title` and `Family_Size_Grouped` are object type, and `Age` and `Fare` features are category type. They are converted to numerical type with `LabelEncoder`. `LabelEncoder` basically labels the classes from **0** to **n**. This process is necessary for Machine Learning algorithms to learn from those features.

In [ ]:
non_numeric_features = ['Embarked', 'Sex', 'Deck', 'Title', 'Family_Size_Grouped', 'Age', 'Fare']

for df in dfs:
    for feature in non_numeric_features:        
        df[feature] = LabelEncoder().fit_transform(df[feature])

>>> #### **2.5.2. One-Hot Encoding the Categorical Features**
The categorical features (`Pclass`, `Sex`, `Deck`, `Embarked`, `Title`) are converted to one-hot encoded features with `OneHotEncoder`. `Age` and `Fare` features are not converted because they are ordinal unlike the former ones.

In [ ]:
cat_features = ['Pclass', 'Sex', 'Deck', 'Embarked', 'Title', 'Family_Size_Grouped']
encoded_features = []

for df in dfs:
    for feature in cat_features:
        encoded_feat = OneHotEncoder().fit_transform(df[feature].values.reshape(-1, 1)).toarray()
        n = df[feature].nunique()
        cols = ['{}_{}'.format(feature, n) for n in range(1, n + 1)]
        encoded_df = pd.DataFrame(encoded_feat, columns=cols)
        encoded_df.index = df.index
        encoded_features.append(encoded_df)

df_train = pd.concat([df_train, *encoded_features[:6]], axis=1)
df_test = pd.concat([df_test, *encoded_features[6:]], axis=1)

>> ### **2.6. Conclusion**
* `Age` and `Fare` features are binned. Binning helped dealing with outliers and it revealed some homogeneous groups in those features.
* `Family_Size` is created with the interaction between `Parch` and `SibSp` features.
* `Name` feature is very useful. First, `Title` and `Is_Married` features are created from the title prefix in the name values. Second, `Family_Survival_Rate` and `Family_Survival_Rate_NA`  features are created by target encoding the surname of the passengers.
* `Survival_Rate` feature is created by averaging the `Family_Survival_Rate` and `Ticket_Survival_Rate` features.
* Finally, the non-numeric type features are label encoded and categorical features are one-hot encoded.
* Created **5** new features (`Family_Size`, `Title`, `Is_Married`, `Survival_Rate` and `Survival_Rate_NA`) and dropped the useless features after encoding.

In [ ]:
df_all = concat_df(df_train, df_test)
drop_cols = ['Deck', 'Embarked', 'Family', 'Family_Size', 'Family_Size_Grouped', 'Survived',
             'Name', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Ticket', 'Title',
            'Ticket_Survival_Rate', 'Family_Survival_Rate', 'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA']

df_all.drop(columns=drop_cols, inplace=True)

df_all.head()

> ## **3. Machine Learning**

In [ ]:
X_train = df_train.drop(columns=drop_cols)
y_train = df_train['Survived']
X_test = df_test.drop(columns=drop_cols)

print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))

>> ### **3.1. Random Forest Classifier**

In [ ]:
clf = RandomForestClassifier(criterion='gini', 
                            n_estimators=1100,
                            max_depth=5,
                            min_samples_split=4,
                            min_samples_leaf=5,
                            max_features='auto',
                            oob_score=True, 
                            random_state=SEED,
                            n_jobs=-1,
                            verbose=1) 

clf.fit(StandardScaler().fit_transform(X_train), y_train)
print('RandomForestClassifier oob score: {}'.format(clf.oob_score_))

>> ### **3.2. Evaluation**

In [ ]:
df_feature_importance = pd.DataFrame()
df_feature_importance['feature'] = X_train.columns
df_feature_importance['importance'] = clf.feature_importances_

plt.figure(figsize=(15, 20))
sns.barplot(x='importance', y='feature', data=df_feature_importance.sort_values(by='importance', ascending=False))
plt.title('Random Forest Classifier Feature Importance')
plt.show()

In [ ]:
y_pred = clf.predict(StandardScaler().fit_transform(X_test)).astype(int)

>> ### **3.3. Submission**

In [ ]:
submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df['Survived'] = y_pred
submission_df.to_csv('submissions.csv', header=True, index=False)
submission_df.head(10)